# Running Charged Defect Calculation


In [7]:
# First make sure that the defect analysis add-on is installed
# If the defect package is not installed, uncomment the following line
# !pip install pymatgen-analysis-defects;

In [12]:
from pymatgen.ext.matproj import MPRester
from pymatgen.analysis.defects.generators import (
    ChargeInterstitialGenerator,
    SubstitutionGenerator,
    VacancyGenerator,
)
import itertools




## Generate the Defect Objects from MP Charge Density

The vacancies and substitutional defects can be generated from the structure alone.

But interstitial are best generated using the electronic charge density.

We can download the charge density from the Materials Project and use it to generate all of the native point defects.

In [ ]:

# Make sure that you have the full featured API installed 
# via `pip install mp-api`
mpr = MPRester() # or add your own API key as an argument here
chgcar = mpr.get_charge_density_from_material_id("mp-804")


Retrieving TaskDoc documents: 100%|██████████| 6/6 [00:00<00:00, 167772.16it/s]


You can get the enumerate over different speices involved in the
vacancy, interstitial and substitutions using the helper functions
below.

In [13]:
def _get_elements(struct) -> tuple[tuple[str], ...]:
    """Get the elements involved in the structures."""
    s_atoms = sorted(atom.symbol for atom in struct.elements)
    return tuple((aa_,) for aa_ in s_atoms)


def _get_subs(struct) -> tuple[dict[str, str], ...]:
    """Get native substitutions."""
    s_atoms = sorted(atom.symbol for atom in struct.elements)
    return tuple(
        itertools.chain.from_iterable(
            ({a: b}, {b: a}) for a, b in itertools.combinations(s_atoms, 2)
        )
    )

print(_get_elements(chgcar.structure))
print(_get_subs(chgcar.structure))

(('Ga',), ('N',))
({'Ga': 'N'}, {'N': 'Ga'})


The different defect generators are available in the `pymatgen.analysis.defects` module each with their own set of tolerances parameters.

Once those are configured,

In [16]:
VGEN = VacancyGenerator(symprec = 0.01)
IGEN = ChargeInterstitialGenerator(max_insertions=3)
SGEN = SubstitutionGenerator(angle_tolerance=5)


In [17]:
def get_defects(chgcar, max_iter: int = 3):
    """Generate the defects for a chgcar.
    
    Args:
        chgcar: The chgcar object.
        max_iter: The maximum number of defects of each type to generate.
    
    Returns:
        A generator of (defect, index) pairs.
    """
    tup_el = _get_elements(chgcar.structure)
    tup_sub = _get_subs(chgcar.structure)
    for sub_d in tup_sub:
        for ii, defect in enumerate(SGEN.generate(chgcar.structure, sub_d)):
            if ii < max_iter:
                yield defect, ii

    for el in tup_el:
        for ii, defect in enumerate(VGEN.generate(chgcar.structure, el)):
            if ii < max_iter:
                yield defect, ii

    for el in tup_el:
        for ii, defect in enumerate(
            IGEN.generate(
                chgcar,
                el,
            )
        ):
            if ii < max_iter:
                yield defect, ii

In [18]:
for defect, defect_index in get_defects(chgcar):
    print(defect, defect_index)

N subsitituted on the Ga site at at site #0 0
Ga subsitituted on the N site at at site #2 0
Ga Vacancy defect at site #0 0
N Vacancy defect at site #2 0
Ga intersitial site at [0.00,0.00,0.20] 0
Ga intersitial site at [0.35,0.65,0.69] 1
N intersitial site at [0.00,0.00,0.20] 0
N intersitial site at [0.35,0.65,0.69] 1


These generators find 8 defects in the structure.
2 vacancies, 2 substitutions, and 4 interstitials.


In [19]:
from atomate2.vasp.flows.defect import FormationEnergyMaker
from atomate2.vasp.flows.mp import MP24DoubleRelaxMaker, MP24StaticMaker, MP24RelaxMaker
from atomate2.vasp.sets.defect import SPECIAL_KPOINT
from pymatgen.io.vasp.sets import MP24RelaxSet

DEFECT_RELAX_SC = MP24RelaxMaker(
        input_set_generator=MP24RelaxSet(
            use_structure_charge=True,
            user_incar_settings={ "ISIF": 2 }
        ),
        task_document_kwargs={"store_volumetric_data": ["locpot"]},
    )
BULK_RELAX_SC = MP24DoubleRelaxMaker()

maker = FormationEnergyMaker(
    bulk_relax_maker = MP24DoubleRelaxMaker(),
    defect_relax_maker = DEFECT_RELAX_SC,
    collect_defect_entry_data = True,
)


In [ ]:
for defect, defect_index in get_defects(chgcar):
    flow = maker.make(defect, defect_index=defect_index)
    # submit each flow to the queue and analyze the results


Flow(name='formation energy', uuid='2467c5f3-79d5-49b3-87e3-15d501644405')
1. Job(name='bulk supercell', uuid='894c2c65-0226-48cb-9594-571a761d4f95')
2. Job(name='spawn_defect_q_jobs', uuid='1f7008e4-b532-4102-9491-75a9f82c69a9')
3. Job(name='get_defect_entry', uuid='e4a2c3a6-38d7-47a3-9970-85fd8a6bb867')
Flow(name='formation energy', uuid='1831b043-0fea-4839-ab3d-0060d424416f')
1. Job(name='bulk supercell', uuid='3b5fa92f-1a8a-4b59-894a-92e2735e78e0')
2. Job(name='spawn_defect_q_jobs', uuid='3434e760-7a9e-4200-bc5b-bbd0f7128e5c')
3. Job(name='get_defect_entry', uuid='b2d418ee-ed5d-4e58-aec6-785b6b349914')
Flow(name='formation energy', uuid='fac82394-bf86-4cb0-9e39-6f646bf87916')
1. Job(name='bulk supercell', uuid='cf65035e-a274-4367-9f65-e048be18401c')
2. Job(name='spawn_defect_q_jobs', uuid='2608b45c-4419-49e0-bbb4-80187366852f')
3. Job(name='get_defect_entry', uuid='62854c46-70dd-4553-939d-fbbe6ad53651')
Flow(name='formation energy', uuid='6265f2b3-e568-472a-be39-b03852ba1d9f')
1. J